In [100]:
import pandas as pd
import threading
import time

In [102]:
df_list_transcripts = pd.read_csv("df_list_transcripts.csv")
print(len(df_list_transcripts))
print()
df_inter_calls_data = pd.read_csv("df_inter_calls_data.csv")
print(len(df_inter_calls_data))
print()
# print(len(set(df_list_transcripts.ID)-set(df_inter_calls_data.CONTACT_ID)))
# print()
diff2 = df_list_transcripts[~df_list_transcripts.ID.isin(df_inter_calls_data.CONTACT_ID)]
print(len(diff2))

1598

9152

51


In [103]:
def process_data():
    """Reads data, computes the difference, and prints results."""
    df_list_transcripts = pd.read_csv("df_list_transcripts.csv")
    df_inter_calls_data = pd.read_csv("df_inter_calls_data.csv")

    print(f"Total Transcripts: {len(df_list_transcripts)}")
    print(f"Total Processed Calls: {len(df_inter_calls_data)}")

    # Find unprocessed calls
    diff2 = df_list_transcripts[~df_list_transcripts.ID.isin(df_inter_calls_data.CONTACT_ID)]
    print(f"Unprocessed Calls: {len(diff2)}\n")

def auto_refresh(interval=300):
    """Runs the process every 'interval' seconds."""
    while True:
        print("\n🔄 Auto-refreshing data...\n")
        process_data()
        time.sleep(interval)

# Start auto-refreshing in a separate thread
refresh_thread = threading.Thread(target=auto_refresh, daemon=True)
refresh_thread.start()

# Allow manual execution on demand
while True:
    input("\nPress Enter to refresh manually...\n")
    process_data()


🔄 Auto-refreshing data...

Total Transcripts: 1598
Total Processed Calls: 9154
Unprocessed Calls: 49




Press Enter to refresh manually...
 


Total Transcripts: 1598
Total Processed Calls: 9161
Unprocessed Calls: 42




Press Enter to refresh manually...
 


Total Transcripts: 1598
Total Processed Calls: 9170
Unprocessed Calls: 33



...

Total Transcripts: 1598
Total Processed Calls: 9170
Unprocessed Calls: 33




Press Enter to refresh manually...
 


Total Transcripts: 1598
Total Processed Calls: 9170
Unprocessed Calls: 33



KeyboardInterrupt: Interrupted by user


🔄 Auto-refreshing data...

Total Transcripts: 1598
Total Processed Calls: 9170
Unprocessed Calls: 33


🔄 Auto-refreshing data...

Total Transcripts: 1598
Total Processed Calls: 9170
Unprocessed Calls: 33



In [78]:
def list_new_transcripts_from_folderlist(
    aws_access_key,
    aws_secret_key,
    source_bucket,
    custom_location,
    folderlist
):
    """
    Fetch audio file metadata from S3 folders using pagination.
    """

    s3_client = boto3.client(
        's3',
        aws_access_key_id=aws_access_key,
        aws_secret_access_key=aws_secret_key
    )

    # logger.info("Fetching New Transcripts to process...")
    all_files = []

    # Fetch files from each folder in the list
    for folder in folderlist:
        try:
            paginator = s3_client.get_paginator('list_objects_v2')
            pages = paginator.paginate(Bucket=source_bucket, Prefix=folder)

            for page in pages:
                for obj in page.get('Contents', []):
                    file_path = obj['Key']
                    s3_ts = obj['LastModified']

                    # Skip non-JSON files
                    if not file_path.endswith('.json'):
                        continue

                    call_id = file_path.split('/')[-1].split("_analysis_")[0]
                    call_timestamp = pd.to_datetime(file_path.split('analysis_')[-1].split('.')[0].replace('Z', ""))

                    all_files.append({
                        'File': file_path,
                        'ID': call_id,
                        'File_Timestamp': call_timestamp,
                        'File_Date': call_timestamp.date().strftime('%Y-%m-%d'),
                        'File_Time': call_timestamp.time().strftime('%H:%M:%S'),
                        'S3_Timestamp': s3_ts,
                        'S3_Date': s3_ts.strftime('%Y-%m-%d'),
                        'S3_Time': s3_ts.strftime('%H:%M:%S')
                    })

        except ClientError as e:
            # logger.error(f"Error accessing S3: {e}")
            continue

    if all_files:
        df_calls_list = pd.DataFrame(all_files).sort_values(['File_Timestamp'], ascending=False)
        df_calls_list['Time_Bin'] = df_calls_list['File_Timestamp'].dt.floor('2h')
    else:
        df_calls_list = pd.DataFrame()

    return df_calls_list

In [79]:
# Get current date
current_date = datetime.now()

# Get dates for last 2 days
num_days = 8
dates_to_check = []
for i in range(num_days):
    check_date = current_date - timedelta(days=i)
    dates_to_check.append(check_date)

# Process each date
folderlist = []
for date in dates_to_check:
    year = str(date.year)
    month = f"{date.month:02d}"
    day = f"{date.day:02d}"
    
    # Construct the prefix for S3 listing
    prefix = f"{s3_transcripts_location}/{year}/{month}/{day}/"
    folderlist.append(prefix)
folderlist[1:]

['Analysis/Voice/2025/03/05/',
 'Analysis/Voice/2025/03/04/',
 'Analysis/Voice/2025/03/03/',
 'Analysis/Voice/2025/03/02/',
 'Analysis/Voice/2025/03/01/',
 'Analysis/Voice/2025/02/28/',
 'Analysis/Voice/2025/02/27/']

In [84]:
df_list_transcripts = list_new_transcripts_from_folderlist(aws_access_key, aws_secret_key, s3_source_bucket, s3_transcripts_location, folderlist[1:])

# logger.info("")
print(f"Transcripts to process: {len(df_list_transcripts)}")
print(df_list_transcripts.groupby(['File_Date']).size().reset_index(name='frequency'))

Transcripts to process: 9381
    File_Date  frequency
0  2025-02-27       2312
1  2025-02-28       2011
2  2025-03-01         65
3  2025-03-03       1950
4  2025-03-04       1445
5  2025-03-05       1598
